In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
from collections import defaultdict 
import time
from pandas import DataFrame

In [ ]:
class NextDoorInfo:
    def __init__(self):
        self.base_url = 'https://nextdoor.com/find-neighborhood/ca/'
        self.all_city_url = []
        self.all_neighborhood_url = []
        self.city_name_pattern = "(?<=city: \").*(?=\",)"
        self.neighbor_name_pattern = "(?<=shortName: \").*(?=\",)"
        self.attributes_pattern = "(?<=attributes: ).*(?=,)"
        self.final_result = []
        self.cursor = 0
        
    def get_base_page(self):
        self.page = requests.get(self.base_url)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        if self.soup != None:
            print("Got Page")
    
    def get_all_city(self):
        self.results = self.soup.find(id = "child_links")
        self.job_elems = self.results.find_all(class_='link')
        pattern = "(?<=city\/).*(?=\-\-ca)"
        for i in self.job_elems:
            self.all_city_url.append(i['href'])
        print("First 5 Cities")
        for n in range(5):
            print (self.all_city_url[n])

    def get_all_neighborhood(self):
        for city_url in self.all_city_url:
            print(city_url)
            temp_url = []
            
            try:
                city_page = requests.get(city_url)
                city_soup = BeautifulSoup(city_page.content, 'html.parser')
                city_results = city_soup.find(id = 'child_links')
                city_job_elems = city_results.find_all(class_='link')

                for i in city_job_elems:
                    temp_url.append(i['href'])
                    
            except Exception:
                print("No Neighbor")
                
            self.all_neighborhood_url.append(temp_url)
            
    def get_all_tags(self):
        for place in range(self.cursor,len(self.all_neighborhood_url)):
            result = []
            
            if self.all_neighborhood_url[place] != 0:
                for neighbor in self.all_neighborhood_url[place]:
                    sub_result = []
                    print(neighbor)
                    neighborhood_page = requests.get(neighbor)
                    neighborhood_soup = BeautifulSoup(neighborhood_page.content, 'html.parser')
                    try:
                        city_name = re.search(self.city_name_pattern,str(neighborhood_soup)).group(0)
                        neighbor_name = re.search(self.neighbor_name_pattern,str(neighborhood_soup)).group(0)
                        attributes = json.loads(re.search(self.attributes_pattern,str(neighborhood_soup)).group(0))
                        sub_result.append(city_name)
                        sub_result.append(self.all_city_url[place])
                        sub_result.append(neighbor_name)
                        sub_result.append(neighbor)
                        sub_result.append(attributes)
                    except AttributeError:
                        sub_result.append("Problem City")
                        sub_result.append(self.all_city_url[place])
                        sub_result.append("Problem Neighbor")
                        sub_result.append(neighbor)
                        sub_result.append("Problem Attributes")
                    result.append(sub_result)
            else:
                sub_result = []
                sub_result.append("City Name")
                sub_result.append(self.all_city_url[place])
                sub_result.append(None)
                sub_result.append(None)
                sub_result.append(None)
                result.append(sub_result)
        
            self.final_result.extend(result)
            self.cursor += 1
        
    def reset_cursor_n_results(self):
        self.cursor = 0
        self.final_result = []
        

Script

In [ ]:
test = NextDoorInfo()
test.get_base_page()
test.get_all_city()
print(len(test.all_city_url))

In [ ]:
test.get_all_neighborhood()
print(len(test.all_neighborhood_url))

In [ ]:
while test.cursor != len(test.all_city_url):
    test.get_all_tags()
    print("restarting")
    time.sleep(180)
print("done")

In [ ]:
test.final_result

In [ ]:
test.cursor

In [ ]:
df = DataFrame (test.final_result,columns=['city_name','city_url','neighborhood_name','neighborhood_url','tags'])

In [ ]:
df.to_csv(r'F:\170A Data\all_neighbors_modified.csv',index = False, header = True)